##4.1 - Sentiment text classification using Gensim Word2vec

In [1]:
import pandas as pd
import gensim

In [3]:
df_sent = pd.read_csv("https://raw.githubusercontent.com/Mentoria-Financiera-DiploDatos/Entrega3/master/sentimientos/data/feeling_procesado.csv")
print(df_sent.news.size)
df_sent.head(3)

4838


,Unnamed: 0,feeling,news,news_tokens,news_tokens_tagged
0,0,neutral,"According to Gran , the company has no plans t...","['accord', 'gran', 'company', 'plan', 'move', ...","[('accord', 'NN'), ('gran', 'VBD'), ('company'..."
1,1,neutral,Technopolis plans to develop in stages an area...,"['technopolis', 'plan', 'develop', 'stage', 'a...","[('technopolis', 'NN'), ('plan', 'NN'), ('deve..."
2,2,negative,The international electronic industry company ...,"['international', 'electronic', 'industry', 'c...","[('international', 'JJ'), ('electronic', 'JJ')..."


No vamos a tener en cuenta las noticias etiquetadas como 'neutrales' ya que el word2vec que vamos a aplicar de "GoogleNews-vectors-negative300.bin" no va a aportar para esta categoría y la misma va a generar ruido. Al comienzo relizamos todo el proceso incluyendo las noticias 'neutrales' y el resultado no fue muy bueno.

In [4]:
df_sent = df_sent[df_sent.feeling != 'neutral'].reset_index()

# Get names of indexes for which column feeling has value 'neutral'
# index_neutral_news = df_sent[df_sent['feeling'] == 'neutral'].index
# Delete these row indexes from dataFrame
# df_sent.drop(index_neutral_news , inplace=True)
# df_sent.reset_index(inplace=True)

print(df_sent.news.size)
df_sent.head(3)

1966


,index,Unnamed: 0,feeling,news,news_tokens,news_tokens_tagged
0,2,2,negative,The international electronic industry company ...,"['international', 'electronic', 'industry', 'c...","[('international', 'JJ'), ('electronic', 'JJ')..."
1,3,3,positive,With the new production plant the company woul...,"['new', 'production', 'plant', 'company', 'wou...","[('new', 'JJ'), ('production', 'NN'), ('plant'..."
2,4,4,positive,According to the company 's updated strategy f...,"['accord', 'company', 'update', 'strategy', 'y...","[('accord', 'NN'), ('company', 'NN'), ('update..."


In [5]:
# Descargar el archivo embeddings.zip en:
# https://www.kaggle.com/ananyabioinfo/text-classification-using-word2vec/data

url = "../embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

In [6]:
print(embeddings.most_similar('camera', topn = 5))
#print(embeddings.doesnt_match(['apple','banana','flower']))
#embeddings.most_similar(positive = ['king','woman'], negative = ['man'])
print("Cantidad de palabras: ", embeddings.vectors.size)
print("Tamaño del vector para cada palabra: ", embeddings.vector_size)

[('cameras', 0.8131939172744751), ('Wagging_finger', 0.7311819791793823), ('camera_lens', 0.7250816822052002), ('camcorder', 0.7037474513053894), ('Camera', 0.6848659515380859)]
Cantidad de palabras:  900000000
Tamaño del vector para cada palabra:  300


Algoritmo para aplicar el embedding a cada palabra de cada documento (que se encuentre dentro de las palabras con las que cuenta), al finalizar de recorrer todas las palabras de un documento se realiza un promedio para dicha noticia, esto para cada documento del corpus.

In [7]:
docs_vectors = pd.DataFrame() # creating empty final dataframe

for doc in df_sent['news_tokens']: # looping through each document
    temp = pd.DataFrame() # creating a temporary dataframe
    
    for word in doc: # looping through each word of a single document
        try:
            word_vec = embeddings[word] # if word is present in embeddings (goole provides weights associate with words(300)) then proceed
            temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
        except:
            pass
    
    doc_vector = temp.mean() # take the average of each column (w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe

docs_vectors.shape

(1966, 300)

Se termina realizando para cada documento (noticia) una media con todos los vectores de cada palabra que conforman un documento, y este vector promedio se asigna a dicho documento como si fuera el resultado final, estas 300 variables nos deberían aportar una mayor información para poder deducir el sentimiento de la noticia, más información que lo que nos puede aportar el propio texto.

Es por esto que el resultado es una matriz de 1966 documentos x 300 variables.

In [8]:
pd.isnull(docs_vectors).sum().sum()

0

In [9]:
docs_vectors['feeling'] = df_sent['feeling']
docs_vectors = docs_vectors.dropna()

In [10]:
print(pd.isnull(docs_vectors).sum().sum())

print(docs_vectors.shape)
print("Positive: ", docs_vectors[docs_vectors.feeling == 'positive'].feeling.size)
print("Negative: ", docs_vectors[docs_vectors.feeling == 'negative'].feeling.size)

0
(1966, 301)
Positive:  1362
Negative:  604


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(docs_vectors.drop('feeling', axis = 1),
                                                   docs_vectors['feeling'],
                                                   test_size = 0.2,
                                                   random_state = 42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1572, 300), (1572,), (394, 300), (394,))

A partir de aquí vamos a aplicar diferentes modelos, recibiendo como entrada los vectores de 300 variables (x) generados aplicando el word2vec, uno por cada noticia, con su respectiva etiqueta de sentimiento (y).

Esperamos obtener un modelo que prediga el sentimiento de las noticias de forma más acertada que los que obtuvimos en la entrega anterior.

##Ada Boost Classifier

In [12]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators = 20)
model.fit(X_train, y_train)
print(model)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=20, random_state=None)


In [13]:
y_test_pred = model.predict(X_test)

text = "Ada Boost Classifier - Reporte de clasificación del conjunto de testeo"
print(len(text)*"=")
print(text)
print(len(text)*"=")
print(classification_report(y_test, y_test_pred))

Ada Boost Classifier - Reporte de clasificación del conjunto de testeo
              precision    recall  f1-score   support

    negative       0.59      0.31      0.40       124
    positive       0.74      0.90      0.81       270

    accuracy                           0.72       394
   macro avg       0.67      0.61      0.61       394
weighted avg       0.69      0.72      0.68       394



##SGDClassifier: SVM & Regresión Logística

Utilizamos SGDClassifier para probar modelos lineales, y GridSearchCV para probar hiperparametros y realizar Cross-Validation.

In [14]:
param_grid = {
    'loss': [
        'hinge',    # SVM
        'log'       # logistic regression
    ],
    'penalty': ['l2', 'l1', 'elasticnet', None],
    'alpha': [0.00001, 0.0001, 0.001, 0.01]
}

In [21]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=0)
cv = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
cv.fit(X_train, y_train)

best_model = cv.best_estimator_
print(best_model)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


In [22]:
y_test_pred = best_model.predict(X_test)

text = "SGD Classifier - Reporte de clasificación del conjunto de testeo"
print(len(text)*"=")
print(text)
print(len(text)*"=")
print(classification_report(y_test, y_test_pred))

SGD Classifier - Reporte de clasificación del conjunto de testeo
              precision    recall  f1-score   support

    negative       0.55      0.56      0.56       124
    positive       0.80      0.79      0.79       270

    accuracy                           0.72       394
   macro avg       0.67      0.67      0.67       394
weighted avg       0.72      0.72      0.72       394



In [23]:
results = cv.cv_results_
df_result_cv = pd.DataFrame(results)
df_result_cv[df_result_cv.rank_test_score<8][['param_loss', 'param_alpha', 'mean_test_score', 'std_test_score', 'rank_test_score']]

,param_loss,param_alpha,mean_test_score,std_test_score,rank_test_score
7,log,1e-05,0.563609,0.076831,7
8,hinge,0.0001,0.592833,0.078960,1
11,hinge,0.0001,0.585005,0.060471,3
12,log,0.0001,0.573773,0.089658,5
13,log,0.0001,0.584304,0.073269,4
14,log,0.0001,0.573210,0.088846,6
15,log,0.0001,0.589710,0.045837,2


##Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [20, 30, 40, None],
    'n_estimators': [20, 30, 40],
    }

model = RandomForestClassifier(random_state=0)
cv = GridSearchCV(model, param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
cv.fit(X_train, y_train)

best_model = cv.best_estimator_
print(best_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


In [25]:
y_test_pred = best_model.predict(X_test)

text = "Random Forest Classifier - Reporte de clasificación del conjunto de testeo"
print(len(text)*"=")
print(text)
print(len(text)*"=")
print(classification_report(y_test, y_test_pred))

Random Forest Classifier - Reporte de clasificación del conjunto de testeo
              precision    recall  f1-score   support

    negative       0.59      0.38      0.46       124
    positive       0.76      0.88      0.81       270

    accuracy                           0.72       394
   macro avg       0.68      0.63      0.64       394
weighted avg       0.71      0.72      0.70       394



##Conclusión

De los modelos anteriormente probados el que obtuvo mejor resultados tanto en accuracy como en f1 fue el modelo de SVM, luego casí con igual desempeño Regresión Logistica, y por debajo Random Forest seguido de Ada Boost Classifier.

Los resultados de los modelos que hemos obtenido hasta ahora no son mejores a los obtenidos en el entrega anterior, la entrega 3, en la cual no se aplicó ningún tipo de embedding a los textos.

**Enlace de referencia:**

https://www.kaggle.com/ananyabioinfo/text-classification-using-word2vec/notebook